<a href="https://colab.research.google.com/github/JinLeeGG/Survival-Prediction-Model-for-AML-using-Gene-Expression-Data-from-TCGA/blob/main/Data_Analysis_Process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Analysis Process**
- This is data analysis process before investigating the relationship between Early Growth Response1 (EGR1) gene expression leveles and survival outcomes in Acute Myeloid Leukemia (AML) patient using data from Cancer Genome Atlas Research Network

# **Dataset**
- Source: [Genomic and epigenomic landscapes of adult de novo acute myeloid leukemia](https://gdc.cancer.gov/about-data/publications/laml_2012)
- Sample Size: 200 AML patients
- Data Types:
  - Patient Clinical Data
  - RNAseq GAF 2.0 read count



In [79]:
# import libraries
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# **Clinical Dataset**

In [80]:
# Reading clinical dataset
Clinical_df = pd.read_csv('/content/drive/MyDrive/Acute Myeloid Leukemia (TCGA, PanCancer Atlas)/datasets/clinical_patient_laml.tsv', sep='\t')
Clinical_df.head()

,bcr_patient_barcode,acute_myeloid_leukemia_calgb_cytogenetics_risk_category,age_at_initial_pathologic_diagnosis,atra_exposure,bcr_patient_uuid,cumulative_agent_total_dose,cytogenetic_abnormality,cytogenetic_abnormality_other,cytogenetic_analysis_performed_ind,date_of_form_completion,...,prior_diagnosis,prior_hematologic_disorder_diagnosis_indicator,race,steroid_therapy_administered,tissue_source_site,total_dose_units,tumor_tissue_site,vital_status,FISH_test_component,FISH_test_component_percentage_value
0,TCGA-AB-2802,Intermediate/Normal,50,NO,b93cb62a-a7dc-406d-8482-6b51a92ea3c3,0,Normal,[Not Available],YES,2010-12-14,...,NO,NO,WHITE,[Not Available],AB,[Not Available],BONE MARROW,DECEASED,NaN,NaN
1,TCGA-AB-2803,Favorable,61,NO,fb4c9803-3690-4f6a-9402-72a4f36d64d1,0,Normal,t(15;17),YES,2010-12-14,...,NO,NO,WHITE,[Not Available],AB,[Not Available],BONE MARROW,DECEASED,PML-RAR,95
2,TCGA-AB-2804,Intermediate/Normal,30,YES,2fcda6a9-813b-41b2-aae4-ca42c9986287,0,Normal,[Not Available],YES,2010-12-14,...,NO,NO,WHITE,[Not Available],AB,[Not Available],BONE MARROW,LIVING,PML-RAR,9
3,TCGA-AB-2805,Intermediate/Normal,77,NO,ada38f3e-8020-4394-9e7c-50d06dd04769,0,Normal,No,YES,2010-12-14,...,YES,NO,WHITE,[Not Available],AB,[Not Available],BONE MARROW,DECEASED,NaN,NaN
4,TCGA-AB-2806,Favorable,46,NO,e78ff499-037b-450a-ac04-6fb3a9e124a4,4000,t (8;21),[Not Available],YES,2010-12-14,...,NO,NO,WHITE,[Not Available],AB,mg,BONE MARROW,DECEASED,NaN,NaN


# **Informations that we need from clinical dataset:**
  1. bcr_patient_barcode (contains patient's unique id)
  2. vital_status (contains patient's status - DECEASED or LIVING)
  3. days_to_death (contains patient's days to death after dignosis - Deceased patient)
  4. days_to_last_followup (contain's patient's last days to follow up - Alive patients)

## Below shows data inspection from clinical dataset


In [81]:
# Inspecting clinical dataset
Clinical_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 78 columns):
 #   Column                                                                   Non-Null Count  Dtype  
---  ------                                                                   --------------  -----  
 0   bcr_patient_barcode                                                      200 non-null    object 
 1   acute_myeloid_leukemia_calgb_cytogenetics_risk_category                  200 non-null    object 
 2   age_at_initial_pathologic_diagnosis                                      200 non-null    int64  
 3   atra_exposure                                                            200 non-null    object 
 4   bcr_patient_uuid                                                         200 non-null    object 
 5   cumulative_agent_total_dose                                              200 non-null    object 
 6   cytogenetic_abnormality                                                  2

In [82]:
# bcr_patient_barcode contains patient's unique id
Clinical_df['bcr_patient_barcode']

,bcr_patient_barcode
0,TCGA-AB-2802
1,TCGA-AB-2803
2,TCGA-AB-2804
3,TCGA-AB-2805
4,TCGA-AB-2806
...,...
195,TCGA-AB-3007
196,TCGA-AB-3008
197,TCGA-AB-3009
198,TCGA-AB-3011


In [83]:
# days to death (patient has not survived)
Clinical_df['days_to_death']

,days_to_death
0,365
1,792
2,[Not Applicable]
3,576
4,944
...,...
195,[Not Applicable]
196,822
197,576
198,[Not Applicable]


In [84]:
# days to last followup (patient is survived)
Clinical_df['days_to_last_followup']

,days_to_last_followup
0,[Not Available]
1,[Not Available]
2,2556
3,[Not Available]
4,[Not Available]
...,...
195,1581
196,[Not Available]
197,[Not Available]
198,1885


In [85]:
# Patient's current status
Clinical_df['vital_status']

,vital_status
0,DECEASED
1,DECEASED
2,LIVING
3,DECEASED
4,DECEASED
...,...
195,LIVING
196,DECEASED
197,DECEASED
198,LIVING


In [86]:
# days to last known alive only contains not available
Clinical_df['days_to_last_known_alive'].describe()

,days_to_last_known_alive
count,200
unique,1
top,[Not Available]
freq,200


## **Creating a new metadata "Observation period"**
  - In order to analysis patient's survival rates, we need to know patient's total survival dates.
  - From vital_status, we know that whether patient is Living or Deceased.
      - If patient is deceased, we take days from "days_to_death"
      - If patient is living, we take days from "days_to_last_known_alive"
      - Combine these datas and create a new metadata called "Observation period"

In [87]:
# Convert 'days_to_death' into numeric type (non-numeric string turns NaN)
Clinical_df['days_to_death'] = pd.to_numeric(Clinical_df['days_to_death'], errors='coerce')

# Convert 'days_to_last_followup' into numeric type (non-numeric string turns NaN)
Clinical_df['days_to_last_followup'] = pd.to_numeric(Clinical_df['days_to_last_followup'], errors='coerce')

# Creating new metatdata 'Observation Period'
Clinical_df['Observation Period'] = np.where(
    Clinical_df['vital_status'] == 'DECEASED', # If status is DECEASED
    Clinical_df['days_to_death'], # If True
    Clinical_df['days_to_last_followup'] # if False
)

# Checking Observation Period
Clinical_df['Observation Period'].head(20)

,Observation Period
0,365.0
1,792.0
2,2556.0
3,576.0
4,944.0
5,180.0
6,2861.0
7,62.0
8,31.0
9,243.0


In [88]:
Clinical_df['Observation Period'].describe()

,Observation Period
count,186.000000
mean,560.618280
std,589.761637
min,0.000000
25%,152.250000
50%,365.000000
75%,753.500000
max,2861.000000


In [89]:
# How many patiant does not have observation period?
nan_count = Clinical_df['Observation Period'].isna().sum()
print(nan_count)

14


## **Creating a new metadata 'Status'**
  - Currently 'vital_status' is a object type (LIVING or DECEASED)
  - We need to convert this into numeric (1 if the patient is dead and 0 if alive).




In [90]:
Clinical_df['Status'] = np.where(
    Clinical_df['vital_status'] == 'DECEASED',
    1, # if true
    0 # if false
)

# Checking
Clinical_df['Status'].head(20)

,Status
0,1
1,1
2,0
3,1
4,1
5,1
6,0
7,1
8,1
9,1


In [91]:
Clinical_df['Status'].describe()

,Status
count,200.000000
mean,0.665000
std,0.473175
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [92]:
# Check for how many 0s(Alive) and 1s(Dead)
counts = Clinical_df['Status'].value_counts()
counts

,count
Status,
1,133
0,67


In [93]:
# Check for final Clinical Dataframe
Clinical_df

,bcr_patient_barcode,acute_myeloid_leukemia_calgb_cytogenetics_risk_category,age_at_initial_pathologic_diagnosis,atra_exposure,bcr_patient_uuid,cumulative_agent_total_dose,cytogenetic_abnormality,cytogenetic_abnormality_other,cytogenetic_analysis_performed_ind,date_of_form_completion,...,race,steroid_therapy_administered,tissue_source_site,total_dose_units,tumor_tissue_site,vital_status,FISH_test_component,FISH_test_component_percentage_value,Observation Period,Status
0,TCGA-AB-2802,Intermediate/Normal,50,NO,b93cb62a-a7dc-406d-8482-6b51a92ea3c3,0,Normal,[Not Available],YES,2010-12-14,...,WHITE,[Not Available],AB,[Not Available],BONE MARROW,DECEASED,NaN,NaN,365.0,1
1,TCGA-AB-2803,Favorable,61,NO,fb4c9803-3690-4f6a-9402-72a4f36d64d1,0,Normal,t(15;17),YES,2010-12-14,...,WHITE,[Not Available],AB,[Not Available],BONE MARROW,DECEASED,PML-RAR,95,792.0,1
2,TCGA-AB-2804,Intermediate/Normal,30,YES,2fcda6a9-813b-41b2-aae4-ca42c9986287,0,Normal,[Not Available],YES,2010-12-14,...,WHITE,[Not Available],AB,[Not Available],BONE MARROW,LIVING,PML-RAR,9,2556.0,0
3,TCGA-AB-2805,Intermediate/Normal,77,NO,ada38f3e-8020-4394-9e7c-50d06dd04769,0,Normal,No,YES,2010-12-14,...,WHITE,[Not Available],AB,[Not Available],BONE MARROW,DECEASED,NaN,NaN,576.0,1
4,TCGA-AB-2806,Favorable,46,NO,e78ff499-037b-450a-ac04-6fb3a9e124a4,4000,t (8;21),[Not Available],YES,2010-12-14,...,WHITE,[Not Available],AB,mg,BONE MARROW,DECEASED,NaN,NaN,944.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,TCGA-AB-3007,Favorable,35,NO,d3988699-70d6-43e1-b84b-9e38b4d2d2b1,[Not Available],Normal|del (7q) / 7q-|Trisomy 8|t (15;17),NO,YES,2010-12-14,...,WHITE,[Not Available],AB,[Not Available],BONE MARROW,LIVING,PML-RAR|CBF-Beta|-7 or del(7q),15|0|96,1581.0,0
196,TCGA-AB-3008,Intermediate/Normal,22,NO,898a092e-89fe-4010-afea-14c605f99481,8000,Normal,[Not Available],YES,2010-12-14,...,WHITE,[Not Available],AB,mg,BONE MARROW,DECEASED,PML-RAR|MLL|CBF-Beta,3|0|1,822.0,1
197,TCGA-AB-3009,Intermediate/Normal,23,NO,c9b92f8f-4599-47d1-9d12-31e42166a091,[Not Available],Normal,[Not Available],YES,2010-12-14,...,[Not Available],[Not Available],AB,[Not Available],BLOOD,DECEASED,NaN,NaN,576.0,1
198,TCGA-AB-3011,Intermediate/Normal,21,NO,e08f84fe-0013-4734-9966-cd734e6fedc5,[Not Available],Normal,[Not Available],YES,2010-12-14,...,WHITE,[Not Available],AB,[Not Available],BONE MARROW,LIVING,PML-RAR|MLL|TEL-AML 1,0|1|1,1885.0,0


# **RNA Expression Dataset**

In [94]:
# Reading RNA Expression Dataset
gene_df = pd.read_csv('/content/drive/MyDrive/Acute Myeloid Leukemia (TCGA, PanCancer Atlas)/datasets/laml.rnaseq.179_v1.0_gaf2.0_read_count_matrix.txt.tcgaID.txt', sep='\t')

gene_df

,GeneID,TCGA-AB-2803,TCGA-AB-2807,TCGA-AB-2963,TCGA-AB-2826,TCGA-AB-2867,TCGA-AB-2818,TCGA-AB-2808,TCGA-AB-2853,TCGA-AB-2854,...,TCGA-AB-2949,TCGA-AB-2981,TCGA-AB-2999,TCGA-AB-2896,TCGA-AB-2952,TCGA-AB-2920,TCGA-AB-2841,TCGA-AB-2811,TCGA-AB-2979,TCGA-AB-2977
0,?|100132510_calculated,14.00,4.00,31.82,32.00,5.00,0.00,0.00,0.00,30.42,...,19.54,56.00,22.00,17.00,3.00,0.00,15.00,15.00,0.90,36.00
1,?|100134860_calculated,339.42,382.66,198.44,113.44,218.28,177.32,233.36,268.30,178.88,...,684.04,238.28,540.18,837.32,560.84,663.38,783.04,1056.46,853.70,591.32
2,?|10357_calculated,5.00,2.00,8.98,26.12,23.66,5.00,9.20,14.00,3.98,...,1.00,4.74,7.90,5.80,5.36,0.00,6.00,2.00,2.50,20.46
3,?|10431_calculated,1638.14,788.50,2078.18,1253.20,633.58,1065.06,1241.54,1658.58,1464.00,...,975.66,1087.08,2183.28,1331.26,1168.14,1488.88,1343.04,4892.32,1009.80,2349.76
4,?|114130_calculated,23.72,980.76,60.28,35.76,203.26,172.26,123.72,48.52,28.86,...,197.40,105.52,119.48,85.06,143.40,339.22,149.18,212.96,156.58,54.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20437,ZYX|7791_calculated,9613.40,7733.44,17099.60,21996.50,20587.60,22100.10,10197.40,13655.20,14063.70,...,15616.70,43075.40,13611.50,19541.50,42372.30,34502.20,64440.40,165176.00,56408.90,47559.30
20438,ZZEF1|23140_calculated,5332.46,9364.42,9210.82,6321.28,8703.40,9577.96,8040.82,8076.90,9366.36,...,10947.50,15371.80,8626.32,8168.92,11208.60,7845.28,9837.74,9715.26,13772.90,5357.90
20439,ZZZ3|26009_calculated,2452.22,2986.18,4788.52,2025.34,3722.32,2884.50,3697.18,5077.84,2072.30,...,4373.78,2374.28,2821.74,3226.08,3098.56,2486.90,1913.44,1419.48,2620.20,1045.34
20440,psiTPTE22|387590_calculated,33.00,51.88,7.00,140.18,583.76,15.00,47.70,5.04,33.96,...,31.84,10.98,67.00,26.50,187.78,36.94,833.10,25.54,74.98,2.00


# I**nspecting RNA expression dataframe**
 - Contains 20422 genes information (rows)
    - Format:
      - {gene_name}|{Id}_calculated
      - ? : unknown id
 - Contains 179 patient cases (columns)
    - Format:
      - TCGA-AB-2803 (Patient's unique id)

In [95]:
# Inspecting gene dataframe
#   - Contains 20422 genes information (rows)
#   - Contains 179 patient cases (columns)
gene_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20442 entries, 0 to 20441
Columns: 180 entries, GeneID to TCGA-AB-2977
dtypes: float64(179), object(1)
memory usage: 28.1+ MB


In [96]:
# Inspecting number of unknown Gene ID
Unknown_gene_count = gene_df['GeneID'].str.startswith('?').sum()
print(f'Number of unknown gene Id: {Unknown_gene_count}')

Number of unknown gene Id: 123


In [97]:
# Removing unknown gene ID from dataframe

gene_df = gene_df[~gene_df['GeneID'].str.startswith('?')]
gene_df['GeneID']

,GeneID
123,A1BG-AS|503538_calculated
124,A1BG|1_calculated
125,A1CF|29974_calculated
126,A2LD1|87769_calculated
127,A2ML1|144568_calculated
...,...
20437,ZYX|7791_calculated
20438,ZZEF1|23140_calculated
20439,ZZZ3|26009_calculated
20440,psiTPTE22|387590_calculated


In [98]:
# 1. Filter for rows containing the '|' character
gene_df = gene_df[gene_df['GeneID'].str.contains('\|', na=False)].copy()

# Removing "_calculated" from GeneID
gene_df['GeneID'] = gene_df['GeneID'].str.replace('_calculated', '')

# reset index
gene_df = gene_df.reset_index(drop=True)

# Checking
gene_df

,GeneID,TCGA-AB-2803,TCGA-AB-2807,TCGA-AB-2963,TCGA-AB-2826,TCGA-AB-2867,TCGA-AB-2818,TCGA-AB-2808,TCGA-AB-2853,TCGA-AB-2854,...,TCGA-AB-2949,TCGA-AB-2981,TCGA-AB-2999,TCGA-AB-2896,TCGA-AB-2952,TCGA-AB-2920,TCGA-AB-2841,TCGA-AB-2811,TCGA-AB-2979,TCGA-AB-2977
0,A1BG-AS|503538,792.14,1095.44,425.60,363.22,505.22,772.74,570.74,460.62,775.30,...,901.26,670.12,1330.06,506.64,1015.04,1213.56,1274.18,436.34,1417.64,963.22
1,A1BG|1,1139.18,1121.68,322.96,274.50,379.46,599.24,531.26,465.74,723.88,...,801.48,412.22,2261.20,512.84,786.04,1124.96,1311.62,806.68,1446.26,1226.78
2,A1CF|29974,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,A2LD1|87769,194.50,111.06,225.60,92.92,143.86,278.46,123.08,274.40,195.24,...,207.56,269.26,173.72,219.28,171.38,242.20,184.98,242.34,107.64,250.72
4,A2ML1|144568,24.36,11.08,58.84,12.06,17.30,22.10,21.64,34.30,30.32,...,33.12,59.74,25.68,60.42,55.42,89.10,22.82,5.10,24.22,11.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20314,ZYX|7791,9613.40,7733.44,17099.60,21996.50,20587.60,22100.10,10197.40,13655.20,14063.70,...,15616.70,43075.40,13611.50,19541.50,42372.30,34502.20,64440.40,165176.00,56408.90,47559.30
20315,ZZEF1|23140,5332.46,9364.42,9210.82,6321.28,8703.40,9577.96,8040.82,8076.90,9366.36,...,10947.50,15371.80,8626.32,8168.92,11208.60,7845.28,9837.74,9715.26,13772.90,5357.90
20316,ZZZ3|26009,2452.22,2986.18,4788.52,2025.34,3722.32,2884.50,3697.18,5077.84,2072.30,...,4373.78,2374.28,2821.74,3226.08,3098.56,2486.90,1913.44,1419.48,2620.20,1045.34
20317,psiTPTE22|387590,33.00,51.88,7.00,140.18,583.76,15.00,47.70,5.04,33.96,...,31.84,10.98,67.00,26.50,187.78,36.94,833.10,25.54,74.98,2.00


# Saving processed data

In [99]:
os.makedirs('processed_datas', exist_ok=True)

path1 = os.path.join('processed_datas', 'cleaned_clinical_df.csv')
path2 = os.path.join('processed_datas', 'cleaned_gene_df.csv')

Clinical_df.to_csv(path1, index=False)
gene_df.to_csv(path2, index=False)

print(f"DataFrames successfully saved to processed_datas")

DataFrames successfully saved to processed_datas
